# Домашнее задание «Функции и работа с данными»


In [1]:
import pandas as pd
import re

### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

    оценка 2 и ниже — низкий рейтинг;
    оценка 4 и ниже — средний рейтинг;
    оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец class.

In [2]:
#читаем файл с рейтингами
df_ratings = pd.read_csv('ml-latest-small_/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
df_ratings_m = pd.pivot_table(df_ratings, index='movieId', values='rating', aggfunc='mean').reset_index()
df_ratings_m.head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857


In [4]:
def rat_class(rating):
    """Определение класса фильма по сред. оценке рейтинга"""
    if rating <= 2:
        rating_class = 'низкий рейтинг'
    elif rating >= 4.5:
        rating_class = 'высокий рейтинг'
    else:
        rating_class = 'средний рейтинг'
    return rating_class

In [5]:
#определяем класс рейтинга фильма
df_ratings_m['class'] = df_ratings_m['rating'].apply(rat_class)

In [6]:
#df_ratings_m.sort_values(by='rating').head()
df_ratings_m.head()

,movieId,rating,class
0,1,3.872470,средний рейтинг
1,2,3.401869,средний рейтинг
2,3,3.161017,средний рейтинг
3,4,2.384615,средний рейтинг
4,5,3.267857,средний рейтинг


In [7]:
#читаем файл с названими фильмов
df_movies = pd.read_csv('ml-latest-small_/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
#добавляем название фильма к df_ratings_m
df_ratings_final = df_ratings_m.merge(df_movies,left_on='movieId', right_on='movieId', 
                                      how='inner')[['movieId', 'title', 'rating', 'class']]

In [9]:
df_ratings_final.head()

,movieId,title,rating,class
0,1,Toy Story (1995),3.872470,средний рейтинг
1,2,Jumanji (1995),3.401869,средний рейтинг
2,3,Grumpier Old Men (1995),3.161017,средний рейтинг
3,4,Waiting to Exhale (1995),2.384615,средний рейтинг
4,5,Father of the Bride Part II (1995),3.267857,средний рейтинг


### Задание 2

Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}
```

Результат классификации запишите в отдельный столбец region.

In [10]:
#читаем файл с названими фильмов
df_keywords = pd.read_csv('ml-latest-small_/keywords.csv')
df_keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [11]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}
geo_data

{'Центр': ['москва', 'тула', 'ярославль'],
 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [12]:
def geo_class(row):
    row_list = row.split(' ')
    
    region = 'undefined' #по умолчанию регион undefined
    
    for i in row_list:
        for k, v in geo_data.items():
            if i in v:
                region = k #но если есть совпадения с геодата регион изменяется

    return region

In [13]:
#применяем функцию geo_class к столбцу keyword
df_keywords['region'] = df_keywords['keyword'].apply(geo_class)

In [14]:
df_keywords.head(130)

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
125,школьный портал,982692,undefined
126,вк моя страница,979105,undefined
127,авито москва,979292,Центр
128,спортмастер,968154,undefined


### Задание 3 (бонусное)

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010 года.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:

        1. для каждой строки пройдите по всем годам списка years;
        2. если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
        3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

Все файлы находятся в архиве «Дополнительные файлы к лекции и дз».

In [15]:
#читаем файл с рейтингами
df_ratings = pd.read_csv('ml-latest-small_/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [16]:
#читаем файл с названими фильмов
df_movies = pd.read_csv('ml-latest-small_/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
#В переменную years запишите список из всех годов с 1950 по 2010 года.
years = list(range(1950, 2011))

type(years[0])

int

In [18]:
def production_year(title):
    '''каждой строке из названия фильма выставляет год выпуска
    если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год'''
    year = re.findall(r'\(19\d*\)|\(20\d*\)', title)
    movie_year = 1900
    
    if year == []:
        movie_year = 1900
    elif int(year[0][1:5]) in years:
        movie_year = int(year[0][1:5])
        
    return movie_year

In [19]:
#Записываем год выпуска фильма в новый столбец ‘year’.
df_movies['year'] = df_movies['title'].apply(production_year)

In [20]:
df_movies.tail(50)

,movieId,title,genres,year
9075,155064,The Neon Demon (2016),Drama|Horror|Mystery,1900
9076,155392,"Hello, My Name Is Doris (2016)",Drama,1900
9077,155611,Life Is Sacred (2014),Documentary,1900
9078,155820,Keanu (2016),Comedy,1900
9079,156025,Ice Age: The Great Egg-Scapade (2016),Adventure|Animation|Children|Comedy,1900
9080,156387,Sing Street (2016),Drama,1900
9081,156607,The Huntsman Winter's War (2016),Action|Adventure|Drama|Fantasy,1900
9082,156609,Neighbors 2: Sorority Rising (2016),Comedy,1900
9083,156726,Hush (2016),Thriller,1900
9084,157200,Money Monster (2016),Drama|Thriller,1900


In [21]:
#Объединяем файлы с рейтингом и фильмами
df_movies_rat = df_movies.merge(df_ratings,left_on='movieId', right_on='movieId', 
                                      how='inner')[['movieId', 'title', 'year', 'rating']]
df_movies_rat.tail(15)

,movieId,title,year,rating
99989,160565,The Purge: Election Year (2016),1900,2.0
99990,160567,Mike & Dave Need Wedding Dates (2016),1900,4.0
99991,160590,Survive and Advance (2013),1900,5.0
99992,160656,Tallulah (2016),1900,3.5
99993,160718,Piper (2016),1900,4.0
99994,161084,My Friend Rockefeller (2015),1900,2.5
99995,161155,Sunspring (2016),1900,0.5
99996,161594,Kingsglaive: Final Fantasy XV (2016),1900,3.0
99997,161830,Body (2015),1900,1.0
99998,161918,Sharknado 4: The 4th Awakens (2016),1900,1.5


In [22]:
#Считаем средний рейтинг всех фильмов для каждого значения столбца ‘year’ и сортируем результат по убыванию рейтинга.
df_movies_years_rat = df_movies_rat.groupby('year').mean('rating').reset_index().sort_values(by='rating', 
                                                                       ascending=False)[['year', 'rating']]
df_movies_years_rat.head()

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
